In [97]:
import numpy as np
import cv2 as cv
import glob
 
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images = glob.glob('IMG/*.jpg')
for index, fname in enumerate(images):
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (9,6), None)
    
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        
        corners = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
    
    # Draw and display the corners
        cv.drawChessboardCorners(img, (9,6), corners, ret)
        name = 'chessboard_with_corners'+str(index)+'.jpg'
        cv.imwrite(name, img)
cv.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print(ret)

print("Camera Matrix is", mtx, "\n")

print("Distortion coef are ", dist, "\n")

print("Rotation vect is ", rvecs, "\n")

print("Translation vect is", tvecs, )




4.870664555007021
Camera Matrix is [[2.90622999e+03 0.00000000e+00 1.52044923e+03]
 [0.00000000e+00 2.90792873e+03 2.13725879e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] 

Distortion coef are  [[ 1.69859674e-01 -6.70996866e-01  1.43321328e-02  8.04213095e-04
   1.14138393e+00]] 

Rotation vect is  (array([[-0.08066426],
       [-0.27695547],
       [ 1.53922238]]), array([[ 0.16106813],
       [-0.17450293],
       [ 1.55742352]]), array([[0.55175357],
       [0.25590824],
       [1.64981595]]), array([[ 0.16114309],
       [-0.53880678],
       [ 1.54240725]]), array([[0.61329861],
       [0.0533045 ],
       [1.74432805]]), array([[ 0.04853108],
       [-0.10114204],
       [ 1.57159251]]), array([[0.62794836],
       [0.07457976],
       [1.69897641]]), array([[0.4725986 ],
       [0.1840978 ],
       [1.63378203]]), array([[-0.31597529],
       [-0.5143839 ],
       [ 1.50278851]]), array([[-0.20889475],
       [-0.40677608],
       [ 1.51869094]]), array([[ 0.02420047],


In [102]:
list_RT = []
for index, rvec in enumerate(rvecs):
    R, _ = cv2.Rodrigues(rvecs[index]) 
    T = tvecs[index]
    RT = np.hstack((R,T))
    list_RT.append(RT)
print(list_RT[0])

[[ 7.41747290e-03 -9.73811236e-01 -2.27237007e-01  3.92965408e+00]
 [ 9.91943302e-01  3.59045903e-02 -1.21488050e-01 -4.51079935e+00]
 [ 1.26465280e-01 -2.24505093e-01  9.66231854e-01  9.87564533e+00]]


In [98]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.634057578804229


In [101]:
import cv2
import numpy as np

images = glob.glob('*.jpg')

# Assuming you have the estimated intrinsic and extrinsic parameters
# intrinsic_mtx: Intrinsic matrix
# dist: Distortion coefficients
# R: Rotation matrix
# T: Translation vector

for index in range(len(images)):
    name = 'chessboard_with_corners'+str(index)+'.jpg'
    img = cv.imread(name)
    #gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
       # Define 3D points for drawing XYZ axes
    axis_length = 2
    points = np.array([[0, 0, 0], [axis_length, 0, 0], [0, axis_length, 0], [0, 0, axis_length]], dtype=np.float32).reshape(-1, 3, 1)
   
    #    Project 3D points onto the image plane
    image_points, _ = cv2.projectPoints(points, rvecs[index], tvecs[index], mtx, dist)
    
    # Draw XYZ axes
    image_points = np.int32(image_points).reshape(-1, 2)
    img = cv2.line(img, tuple(image_points[0]), tuple(image_points[1]), (0, 0, 255), 8)  # X-axis (red)
    img = cv2.line(img, tuple(image_points[0]), tuple(image_points[2]), (0, 255, 0), 8)  # Y-axis (green)
    immg = cv2.line(img, tuple(image_points[0]), tuple(image_points[3]), (255, 0, 0), 8)  # Z-axis (blue)
    
    cube_size = 3
    x,y,z= 2, 2, 3
    # Define cube vertices
    cube_vertices = np.array([[x, y, z],
                           [x+cube_size, y, z],
                           [x+cube_size, y+cube_size, z],
                           [x, y+cube_size, z],
                           [x, y, z-cube_size],
                           [x+cube_size, y, z-cube_size],
                           [x+cube_size, y+cube_size, z-cube_size],
                           [x, y+cube_size, z-cube_size]], dtype=np.float32)

    # Define cube edges
    cube_edges = [(0, 1), (1, 2), (2, 3), (3, 0),
              (4, 5), (5, 6), (6, 7), (7, 4),
              (0, 4), (1, 5), (2, 6), (3, 7)]
    
        # Project 3D cube vertices onto the image plane
    cube_points, _ = cv2.projectPoints(cube_vertices, rvecs[index], tvecs[index], mtx, dist)
    
    # Convert image points to integer
    cube_points = np.int32(cube_points).reshape(-1, 2)
    
    # Draw cube edges
    for edge in cube_edges:
        start_point = tuple(cube_points[edge[0]])
        end_point = tuple(cube_points[edge[1]])
        img = cv2.line(img, start_point, end_point, (255, 255, 255), 8)
        
    name = '3D/3DAxis'+str(index)+'.jpg'
    cv.imwrite(name, img)
cv2.destroyAllWindows()